In [4]:
import os
import matplotlib.pyplot as plt
from osgeo import gdal
import geopandas as gpd
import pandas as pd
import shutil

In [5]:
# define folder paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_products = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Average AAs')

# open rgi df
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [31]:
# get list of all the .tif AAs in the folder
all_tifs_names = os.listdir(folder_products)

# keep only tif file paths
all_tifs_names = [i for i in all_tifs_names if i[-3:]=='tif']

# keep only the rgi ids that are in o2region x
o2region_target = '6'
all_tifs_names = [i for i in all_tifs_names if rgi_gdf[rgi_gdf['RGIId']==i[3:17]]['O2Region'].values[0]==o2region_target ]

# make them into file paths
all_tifs_paths = [os.path.join(folder_products, i) for i in all_tifs_names]

print(len(all_tifs_paths))

1087


In [32]:
# define path for vrt, output location/name
out_path = os.path.join(folder_AGVA, 'Derived products', 'S2', f"Region_0{o2region_target}_2018_2022_AA.tif")
vrt_path = os.path.join(folder_AGVA, 'Derived products', 'S2', 'merged.vrt')

# build vrt dataset
vrt = gdal.BuildVRT(vrt_path, all_tifs_paths, VRTNodata='nan', srcNodata=0)

# Open the VRT dataset
ds = gdal.Open(vrt_path)

# # set nodata to 0
# for band_num in range(1, ds.RasterCount + 1):
#     band = ds.GetRasterBand(band_num)
#     band.SetNoDataValue(0)

# mosaic
gdal.Translate(out_path, ds, outputType=gdal.GDT_Byte, creationOptions = ['PREDICTOR=1','COMPRESS=ZSTD', 'ZSTD_LEVEL=1', "TILED=TRUE"])
ds = None
print("mosaicing done")

# os.remove(vrt_path)

mosaicing done
